# 라이브러리

In [ ]:
import pandas as pd
import numpy as np
import re 

from tqdm.auto import tqdm
from joblib import parallel_backend
import contextlib


In [ ]:
# @contextlib.contextmanager
# def tqdm_joblib(tqdm_object):
#     from joblib import Parallel
#     old_batch_callback = Parallel.BatchCompletionCallBack

#     class TqdmBatchCompletionCallBack(old_batch_callback):
#         def __call__(self, *args, **kwargs):
#             tqdm_object.update(n=self.batch_size)
#             return super().__call__(*args, **kwargs)

#     Parallel.BatchCompletionCallBack = TqdmBatchCompletionCallBack
#     try:
#         yield tqdm_object
#     finally:
#         Parallel.BatchCompletionCallBack = old_batch_callback
#         tqdm_object.close()


# 재무정보_final_v2(16~24) 파일 열기
- 기업 정보만을 위해 열어보는 파일
    - 재무 데이터에 등장한 모든 기업
    - 연도별로 살아 있었던 기업
    - 특허 여부와 무관

In [ ]:
financial_df = pd.read_csv('../../data/재무정보_final_v2(16~24)_수정_최종.csv', encoding="utf-8", encoding_errors='replace', engine="python")
# financial_df.head(5)

---
- 재무 데이터 전처리
---

In [ ]:
# 수정 전: financial_df['매출액']만 처리하던 부분 (cell_id: e58a3433 부근)
# 수정 후: 모든 주요 재무 컬럼에 대해 전처리 수행

financial_cols = [
    '매출액', '영업이익', '당기순이익', '자산총계', '부채총계', 
    '자본총계', '연구개발비', '유형자산_당기', '유형자산_전기', '유형자산_증감'
]

for col in financial_cols:
    if col in financial_df.columns:
        # 1. 문자열로 변환 후 콤마(,) 제거 및 공백 제거
        # 2. '-' 표시를 '0'으로 변경
        financial_df[col] = (
            financial_df[col]
            .astype(str)
            .str.replace(',', '', regex=False)
            .str.replace(' ', '', regex=False)
            .replace('-', '0')
        )
        # 3. 수치형(float)으로 변환, 변환 불가 항목은 NaN 처리 후 0으로 채움
        financial_df[col] = pd.to_numeric(financial_df[col], errors='coerce').fillna(0)

print("재무 데이터 수치형 변환 완료")

In [ ]:
financial_df.columns.tolist()

# 특허정보_final 파일 열기

In [ ]:
patent_final = pd.read_csv('../../data/특허정보_final_v2.csv', encoding="cp949", encoding_errors='replace', engine="python")
# patent_final.head(5)

In [ ]:
patent_final = patent_final.rename(columns={
    'ipcNumber_IPC코드': 'ipcNumber',
    'applicationDate_출원일자': 'applicationDate',
    'astrtCont_초록':'astrtCont',
    'applicationNumber_출원번호':'applicationNumber',
     'registerDate_등록일자':'registerDate',
      'indexNo_일련번호':'indexNo',
 'registerStatus_등록상태':'registerStatus',
 'inventionTitle_발명의명칭':'inventionTitle',
 'applicantName_출원인명':'applicantName'



    
})


In [ ]:
patent_final.columns.tolist()

---
- 특허 데이터 전처리
---

In [ ]:
patent_df = patent_final.copy()

# 출원일 datetime 보장
patent_df['applicationDate'] = pd.to_datetime(patent_df['applicationDate'])

# 출원년도 
patent_df['application_year'] = patent_df['applicationDate'].dt.year


In [ ]:
patent_df['application_year'] = pd.to_numeric(
    patent_df['application_year'],
    errors='coerce'
)


---
- 기업명 정규화
---

In [ ]:
def normalize_company_name(name: str) -> str:
    if pd.isna(name):
        return None

    name = str(name)

    # 1. 소문자 변환
    name = name.lower()

    # 2. 법인 표기 제거
    name = re.sub(r'\(주\)|㈜|주식회사', '', name)

    # 3. 불필요한 공백 제거
    name = re.sub(r'\s+', '', name)

    # 4. 특수문자 제거 (필요 최소)
    name = re.sub(r'[^\w가-힣]', '', name)

    return name


In [ ]:
#=====================================================================================
# 두 데이터에 동일하게 적용 
#=====================================================================================
financial_df['company_norm'] = financial_df['기업명'].apply(normalize_company_name)
patent_df['company_norm'] = patent_df['company_name'].apply(normalize_company_name)

#=====================================================================================
# 매칭 품질 진단
#=====================================================================================

# 재무 기준 기업 수
financial_companies = set(financial_df['company_norm'].dropna())

# 특허 기준 기업 수
patent_companies = set(patent_df['company_norm'].dropna())

print("재무 기업 수:", len(financial_companies))
print("특허 기업 수:", len(patent_companies))
print("교집합 수:", len(financial_companies & patent_companies))


In [ ]:
company_year_df = (
    financial_df[['company_norm', '연도']]
    .drop_duplicates()
    .sort_values(['company_norm', '연도'])
    .reset_index(drop=True)
)


# 특허 피처 생성 함수

In [ ]:
import pandas as pd
import numpy as np

def make_patent_features(company_norm, t, patent_df):
    """
    company_norm × 연도(t) 기준 특허 피처 생성
    - warm-up 적용: t < 2019 → None
    - 누수 방지: t-1 시점까지만 사용
    """

    # ===============================
    # 0. Warm-up 강제
    # ===============================
    t = int(t)
    if t < 2019:
        return None

    # 기업 기준 필터
    df = patent_df[patent_df['company_norm'] == company_norm].copy()

    # ===============================
    # 1. 기간 필터 (누수 방지 핵심)
    # ===============================
    window_5y = df[
        (df['application_year'] >= t - 4) &
        (df['application_year'] <= t - 1)
    ]

    window_recent = df[
        (df['application_year'] >= t - 2) &
        (df['application_year'] <= t - 1)
    ]

    # # 등록률 코호트: t-4 ~ t-3
    # cohort_register = df[
    #     (df['application_year'] >= t - 4) &
    #     (df['application_year'] <= t - 3)
    # ]

    # 피인용 as-of cutoff
    citation_window = window_5y[
        window_5y['applicationDate'] <= pd.Timestamp(f"{t-1}-12-31")
    ].copy()

    # ===============================
    # 2. 피처 계산
    # ===============================
    patent_count_5y = len(window_5y)
    patent_count_recent = len(window_recent)

    ipc_diversity = (
        window_5y['ipcNumber']
        .dropna()
        .astype(str)
        .str[:4]
        .nunique()
        if patent_count_5y > 0 else 0
    )

    # # 등록률: registerDate 기준
    # if len(cohort_register) > 0:
    #     patent_register_rate = (
    #         cohort_register['registerDate'].notna().mean()
    #     )
    # else:
    #     patent_register_rate = 0.0
    # ===============================
    # 등록률 코호트: t-4 ~ t-3
    # ===============================
    VALID_SUCCESS_STATUS = ['등록', '공개']
    cohort_register = df[
        (df['application_year'] >= t - 4) &
        (df['application_year'] <= t - 3)
    ]

    if len(cohort_register) > 0:
        # 전체 출원 수
        total_applications = len(cohort_register)

        # 등록 + 공개 건수
        success_count = cohort_register[
            cohort_register['registerStatus'].isin(['등록', '공개'])
        ].shape[0]

        patent_register_rate = success_count / total_applications
    else:
        patent_register_rate = 0.0


    # -------------------------------
    # 피인용 (기본)
    # -------------------------------
    patent_citation_total = (
        citation_window['피인용 횟수'].sum()
        if len(citation_window) > 0 else 0.0
    )

    patent_citation_avg = (
        patent_citation_total / max(1, patent_count_5y)
    )

    # -------------------------------
    # 🔥 연령 보정 피인용
    # -------------------------------
    if len(citation_window) > 0:
        citation_window['patent_age'] = (
            t - citation_window['application_year']
        ).clip(lower=1)

        citation_window['citation_per_year'] = (
            citation_window['피인용 횟수'] / citation_window['patent_age']
        )

        patent_citation_age_adj_total = (
            citation_window['citation_per_year'].sum()
        )
    else:
        patent_citation_age_adj_total = 0.0

    patent_citation_age_adj_avg = (
        patent_citation_age_adj_total / max(1, patent_count_5y)
    )

    # ===============================
    # 최근 활동 비중
    # ===============================
    patent_recent_ratio = (
        patent_count_recent / patent_count_5y
        if patent_count_5y > 0 else 0.0
    )

    citation_recent = (
        window_recent['피인용 횟수'].sum()
        if len(window_recent) > 0 else 0.0
    )

    citation_recent_ratio = (
        citation_recent / patent_citation_total
        if patent_citation_total > 0 else 0.0
    )

    # ===============================
    # 연도별 특허 수 (증가율 계산용)
    # ===============================
    yearly_counts = (
        window_5y
        .groupby('application_year')
        .size()
        .sort_index()
    )

    if len(yearly_counts) >= 2:
        patent_growth_rate = yearly_counts.pct_change().iloc[-1]
    else:
        patent_growth_rate = 0.0

    if len(yearly_counts) >= 3:
        patent_growth_accel = (
            yearly_counts.pct_change().diff().iloc[-1]
        )
    else:
        patent_growth_accel = 0.0

    # ===============================
    # IPC 구조 변화
    # ===============================
    ipc_old = set(
        window_5y[
            window_5y['application_year'] <= t - 3
        ]['ipcNumber']
        .dropna()
        .astype(str)
        .str[:4]
    )

    ipc_recent = set(
        window_recent['ipcNumber']
        .dropna()
        .astype(str)
        .str[:4]
    )

    if len(ipc_old) > 0:
        new_ipc_ratio = len(ipc_recent - ipc_old) / len(ipc_old)
        recent_ipc_overlap = len(ipc_recent & ipc_old) / len(ipc_old)
    else:
        new_ipc_ratio = 0.0
        recent_ipc_overlap = 0.0

    # ===============================
    # 최근 특허 질
    # ===============================
    citation_per_patent_recent = (
        citation_recent / max(1, patent_count_recent)
    )


    # -------------------------------
    # 특허 0건 플래그
    # -------------------------------
    no_patent_flag = int(patent_count_5y == 0)

    return {
        'company_norm': company_norm,
        '연도': t,
        'patent_count_5y': patent_count_5y,
        'patent_count_recent': patent_count_recent,
        'ipc_diversity': ipc_diversity,
        'patent_register_rate': patent_register_rate,
        'patent_citation_total': patent_citation_total,
        'patent_citation_avg': patent_citation_avg,
        # 🔥 추가 피처
        'patent_citation_age_adj_total': patent_citation_age_adj_total,
        'patent_citation_age_adj_avg': patent_citation_age_adj_avg,
        'no_patent_flag': no_patent_flag,

            # 🔥 변화형 피처
        'patent_recent_ratio': patent_recent_ratio,
        'citation_recent_ratio': citation_recent_ratio,
        'patent_growth_rate': patent_growth_rate,
        'patent_growth_accel': patent_growth_accel,
        'new_ipc_ratio': new_ipc_ratio,
        'recent_ipc_overlap': recent_ipc_overlap,
        'citation_per_patent_recent': citation_per_patent_recent
    }


In [ ]:
patent_feature_rows = []

for row in tqdm(
    company_year_df.itertuples(index=False),
    total=len(company_year_df),
    desc="특허 피처 생성"
):
    feat = make_patent_features(
        company_norm=row.company_norm,
        t=row.연도,
        patent_df=patent_df
    )

    if feat is not None:
        patent_feature_rows.append(feat)

patent_feature_df = pd.DataFrame(patent_feature_rows)


In [ ]:
patent_feature_df['연도'].unique()


In [ ]:
patent_feature_df.to_csv('../../output/feature_engineering_dataset_13_patent.csv', encoding='utf-8-sig')

- 특허 피처 엔지니어링 과정에서 롤링 윈도우의 warm-up 기간을 함수 내부에서 강제하여,
충분한 과거 데이터가 확보되지 않은 연도(2019 이전)가
학습 데이터에 포함되지 않도록 수정하였다.

# 재무 데이터와 결합

In [ ]:
financial_df['연도'] = pd.to_numeric(financial_df['연도'], errors='coerce').astype('Int64')
patent_feature_df['연도'] = pd.to_numeric(patent_feature_df['연도'], errors='coerce').astype('Int64')


In [ ]:
financial_df[['연도']].dtypes, patent_feature_df[['연도']].dtypes


In [ ]:
financial_df['매출액'].isna().mean(), (financial_df['매출액'] == 0).mean()


In [ ]:
financial_agg = (
    financial_df
    .groupby(['company_norm', '연도'], as_index=False)
    .agg({
        '기업명': 'first',
        '사업자등록번호': 'first',
        'crno': 'first',
        'dart_corp_code': 'first',
        'sectors': 'first',
        'field': 'first',

        '매출액': 'sum',
        '영업이익': 'sum',
        '당기순이익': 'sum',
        '자산총계': 'max',
        '부채총계': 'max',
        '자본총계': 'max',
        '연구개발비': 'sum',
        'CAPEX': 'sum',
        '유형자산_당기': 'max',
        '유형자산_전기': 'max',
        '유형자산_증감': 'sum'
    })
)


In [ ]:
financial_agg

In [ ]:
final_df = financial_agg.merge(
    patent_feature_df,
    on=['company_norm', '연도'],
    how='left',
    validate='one_to_one'
)


In [ ]:
len(financial_agg), len(final_df)


In [ ]:
final_df['매출액'].isna().mean()


In [ ]:
final_df

In [ ]:
print('financial_df:',financial_df.columns)
print('patent_feature_df:',patent_feature_df.columns)
print('final_df:',final_df.columns)


In [ ]:
print(len(financial_df), len(final_df))


## 타깃(y) 생성 — “t 정보로 t+1 성장 여부 예측”
- 피처 데이터셋 저장할 때 final_df 로 해줘야 company_norm, 기업명 컬럼 둘 다 나올 수 있음

In [ ]:
final_df['revenue_growth'] = (
    final_df
    .groupby('company_norm')['매출액']
    .pct_change()       # pct_change를 통해서 매출 성장률 계산됨
)


In [ ]:
final_df.head(3)

In [ ]:
final_df = final_df[
    (final_df['연도'] >= 2019) &
    (final_df['연도'] <= 2024)
].copy()


In [ ]:
final_df['연도'].value_counts().sort_index()


In [ ]:
# growth_flag : 같은 연도 안에서 성장률이 상위 30%면 1
final_df['growth_flag'] = (
    final_df
    .groupby('연도')['revenue_growth']
    .transform(lambda x: (x >= x.quantile(0.7)).astype(int))
)


In [ ]:
final_df.groupby('연도')['growth_flag'].mean()


In [ ]:
# y : 올해의 정보로 다음 해에 잘 성장할 기업인지 예측하기 위해서 1년 뒤로 당겨서 만듦
final_df['y'] = (
    final_df
    .groupby('company_norm')['growth_flag']
    .shift(-1)
)


In [ ]:
model_df = final_df.dropna(subset=['y']).copy()
model_df['y'] = model_df['y'].astype(int)


In [ ]:
model_df['연도'].value_counts().sort_index()


In [ ]:
model_df.columns

In [ ]:
final_df.columns

In [ ]:
# 필요한 컬럼들만 남기기
keep_cols = [
    # 식별 / 시점
    'company_norm',
    '연도',
    '기업명',

    # 특허 피처
    'patent_count_5y',
    'patent_count_recent',
    'ipc_diversity',
    'patent_register_rate',
    'patent_citation_total',
    'patent_citation_avg',
    'patent_citation_age_adj_total',
    'patent_citation_age_adj_avg',
    'no_patent_flag',
    'patent_recent_ratio', 'citation_recent_ratio',
       'patent_growth_rate', 'patent_growth_accel', 'new_ipc_ratio',
       'recent_ipc_overlap', 'citation_per_patent_recent'

    # 성장 관련
    # 'revenue_growth',
    # 'growth_flag',
    # 'y'
]

final_df = final_df[keep_cols].copy()


In [ ]:
final_df.to_csv('../../output/feature_engineering_dataset_14_patent.csv', encoding='utf-8-sig')

# 모델링
- 모델 : LogReg, RandomForest, LightBGM, XGBoost
- 검증 방식 : Holdout, GroupKFold, RollingValidation
- 평가지표: PR-AUC, Top-20/50 Precision, F1, Brier
---
> - model_df : feature + y 포함 (정상)
> - 연도 범위 : 2019–2023
> - y : t년 피처 → t+1년 성장 여부(0,1)

# X랑 y에 들어갈 df 불러오기
- 기존에 만든 model_df 사용하지 않고 재무 데이터 정규화된 데이터와 정답 데이터 따로 받은 후 결합

### feature 데이터 

In [ ]:
feature_df = pd.read_csv('../../data/normalized_patent_features_add.csv', encoding="utf-8", encoding_errors='replace', engine="python")
feature_df.head(3)

In [ ]:
feature_df.columns.to_list

### 정답 데이터

In [ ]:
target_df = pd.read_csv('../../data/연도별 성장여부.csv', encoding="utf-8", encoding_errors='replace', engine="python")
target_df.head(3)

In [ ]:
target_long_df = target_df.melt(
    id_vars="기업명",        # 고정할 컬럼
    var_name="연도",        # 새로 생길 연도 컬럼 이름
    value_name="y"          # 타겟 변수
)

target_long_df.head(3)


In [ ]:
target_long_df.to_csv('../../output/연도별_성장여부_컬럼변환.csv', encoding='utf-8-sig')

In [ ]:
# feature 데이터 연도 → int
feature_df["연도"] = feature_df["연도"].astype(int)

# target 데이터 연도 → int
target_long_df["연도"] = target_long_df["연도"].astype(int)


In [ ]:
model_df = feature_df.merge(
    target_long_df,
    left_on=["기업명", "target_year"],
    right_on=["기업명", "연도"],
    how="left"
)

model_df.head(3)

In [ ]:
model_df.columns.tolist()


# 왼쪽 연도  → 연도_x
# 오른쪽 연도 → 연도_y


## Feature / Label 분리

In [ ]:
# -------------------------
# Feature 그룹 정의
# -------------------------

patent_cols = [
    'patent_count_5y',
    'patent_count_recent',
    'ipc_diversity',
    'patent_register_rate',
    'patent_citation_total',
    'patent_citation_avg',
    'patent_citation_age_adj_total',
    'patent_citation_age_adj_avg',
    'no_patent_flag',
    'patent_recent_ratio',
       'citation_recent_ratio', 'patent_growth_rate', 'patent_growth_accel',
       'new_ipc_ratio', 'recent_ipc_overlap', 'citation_per_patent_recent'
]

patent_emb_cols = [f'patent_emb_{i}' for i in range(50)]

#====================================
# 특허 정량 지표만 쓰고 싶을 떄 
X = model_df[patent_cols].copy()

# 임베딩 같이 쓰고 싶을 때
# X = model_df[patent_cols + patent_emb_cols].copy()
#====================================
y = model_df['y'].copy()


## train/test 분리

In [ ]:
# =====================================
# Time-based Train / Test Split
# =====================================

# 1. 최종 테스트 연도 지정
TEST_TARGET_YEAR = 2024

# 2. Train / Test 인덱스 분리
train_idx = model_df['target_year'] < TEST_TARGET_YEAR
test_idx  = model_df['target_year'] == TEST_TARGET_YEAR

# 3. 분리
X_train = X.loc[train_idx].copy()
y_train = y.loc[train_idx].copy()

X_test  = X.loc[test_idx].copy()
y_test  = y.loc[test_idx].copy()

# Time-based split 결과
print("Train target years:", sorted(y_train.index.map(lambda i: model_df.loc[i, 'target_year']).unique()))
print("Test target year:", model_df.loc[test_idx, 'target_year'].unique())
print("Train size:", X_train.shape)
print("Test size:", X_test.shape)

In [ ]:
print("y_test NaN 개수:", y_test.isna().sum())


In [ ]:
# train 데이터
train_df = X_train.copy()
train_df["target"] = y_train
train_df = train_df.dropna(subset=["target"])

X_train = train_df.drop(columns="target")
y_train = train_df["target"]

# test 데이터
test_df = X_test.copy()
test_df["target"] = y_test
test_df = test_df.dropna(subset=["target"])

X_test = test_df.drop(columns="target")
y_test = test_df["target"]


---

- 2019 → 2020 (Train)

- 2020 → 2021 (Train)

- 2021 → 2022 (Train)

- 2022 → 2023 (Train)

- 2023 → 2024 (Test) ✅

👉 미래 정보 섞인 거 없음 (데이터 누수 ❌)

---

- feature 개수 : 9개
- train/test 비율 : 약 8:2

In [ ]:
# 연도 정보 (보조 변수)
years_train = model_df.loc[train_idx, 'target_year'].copy()
years_test  = model_df.loc[test_idx, 'target_year'].copy()

print(sorted(years_train.unique()))
print(sorted(years_test.unique()))


## 공통 평가 함수

In [ ]:
import numpy as np

def top_k_precision(y_true, y_proba, k):
    """
    Top-K Precision 계산
    - y_proba 기준 상위 K개 선택
    """
    if len(y_true) < k:
        k = len(y_true)

    top_k_idx = np.argsort(y_proba)[::-1][:k]
    return y_true.iloc[top_k_idx].mean()


In [ ]:
from sklearn.metrics import (
    f1_score,
    average_precision_score,
    brier_score_loss
)

def evaluate_model(y_true, y_pred, y_proba):
    """
    공통 모델 평가 함수 (성장기업 예측 목적)
    - PR-AUC
    - Top-K Precision (K=20, 50)
    - F1 Score
    - Brier Score
    """

    return {
        # PR-AUC
        "pr_auc": average_precision_score(y_true, y_proba),

        # Top-K Precision
        "top20_precision": top_k_precision(y_true, y_proba, k=20),
        "top50_precision": top_k_precision(y_true, y_proba, k=50),

        # Classification balance
        "f1": f1_score(y_true, y_pred),

        # Probability calibration
        "brier": brier_score_loss(y_true, y_proba)
    }


## K-fold 검증 
- 과적합 있는지 판단하기 위해

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.base import clone
from sklearn.metrics import average_precision_score
import numpy as np

def cross_validate_overfitting_check(
    model,
    X_train,
    y_train,
    n_splits=5,
    random_state=42,
    verbose=True
):
    """
    Train 데이터 내부에서 k-fold로 과적합 진단
    - metric: PR-AUC
    """

    skf = StratifiedKFold(
        n_splits=n_splits,
        shuffle=True,
        random_state=random_state
    )

    pr_auc_scores = []

    for fold, (tr_idx, val_idx) in enumerate(skf.split(X_train, y_train), 1):
        # 🔹 fold마다 모델 복제 (중요)
        fold_model = clone(model)

        X_tr, X_val = X_train.iloc[tr_idx], X_train.iloc[val_idx]
        y_tr, y_val = y_train.iloc[tr_idx], y_train.iloc[val_idx]

        fold_model.fit(X_tr, y_tr)

        y_val_proba = fold_model.predict_proba(X_val)[:, 1]

        score = average_precision_score(y_val, y_val_proba)
        pr_auc_scores.append(score)

        if verbose:
            print(f"Fold {fold} PR-AUC: {score:.4f}")

    return {
        "cv_pr_auc_mean": np.mean(pr_auc_scores),
        "cv_pr_auc_std": np.std(pr_auc_scores),
        "cv_pr_auc_min": np.min(pr_auc_scores),
        "cv_pr_auc_max": np.max(pr_auc_scores)
    }


## 모델 + 하이퍼파라미터 그리드

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

MODELS = {
    "Logistic": (
        LogisticRegression(
            max_iter=2000,
            class_weight="balanced",
            solver="liblinear"
        ),
        {"C": [0.01, 0.1, 1, 10]}
    ),

    "RF": (
        RandomForestClassifier(
            random_state=42,
            class_weight="balanced"
        ),
        {
            "n_estimators": [300, 500],
            "max_depth": [None, 5, 10],
            "min_samples_leaf": [1, 5, 10]
        }
    ),

    "XGB": (
        XGBClassifier(
            objective="binary:logistic",
            eval_metric="aucpr",
            random_state=42,
            use_label_encoder=False
        ),
        {
            "n_estimators": [300, 600],
            "max_depth": [3, 5],
            "learning_rate": [0.03, 0.1],
            "subsample": [0.8, 1.0],
            "colsample_bytree": [0.8, 1.0]
        }
    ),

    "LGBM": (
        LGBMClassifier(
            objective="binary",
            random_state=42
        ),
        {
            "n_estimators": [300, 600],
            "learning_rate": [0.03, 0.1],
            "num_leaves": [31, 63],
            "max_depth": [-1, 10],
            "subsample": [0.8, 1.0],
            "colsample_bytree": [0.8, 1.0]
        }
    )
}

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, average_precision_score

pr_auc_scorer = make_scorer(
    average_precision_score,
    needs_proba=True
)



In [ ]:
import mlflow
import mlflow.sklearn

# 실험 이름 지정
mlflow.set_experiment("growth_prediction_time_based")

results = []
TEST_TARGET_YEAR = 2024

for feature_set_name, feature_cols in FEATURE_SETS.items():
    print(f"\n==============================")
    print(f"🧪 Feature set: {feature_set_name}")

    X_feat = X_all[feature_cols]

    X_train, X_test, y_train, y_test = make_time_based_split_from_xy(
        X=X_feat,
        y=y_all,
        time_series=time_series,
        test_target_year=TEST_TARGET_YEAR
    )

    for model_name, (model, param_grid) in MODELS.items():
        print(f"\n▶ {model_name} tuning")

        with mlflow.start_run(
            run_name=f"{feature_set_name}_{model_name}"
        ):
            # =========================
            # 1. Tag
            # =========================
            mlflow.set_tag("feature_set", feature_set_name)
            mlflow.set_tag("model", model_name)
            mlflow.set_tag(
                "has_patent_embedding",
                "EMB" in feature_set_name
            )

            # =========================
            # 2. Hyperparameter tuning
            # =========================
            grid = GridSearchCV(
                estimator=model,
                param_grid=param_grid,
                scoring=pr_auc_scorer,
                cv=3,
                n_jobs=-1
            )

            grid.fit(X_train, y_train)

            best_model = grid.best_estimator_

            # =========================
            # 3. Prediction
            # =========================
            y_pred = best_model.predict(X_test)
            y_proba = best_model.predict_proba(X_test)[:, 1]

            # =========================
            # 4. Evaluation
            # =========================
            metrics = evaluate_model(y_test, y_pred, y_proba)

            # =========================
            # 5. MLflow logging
            # =========================
            mlflow.log_param("feature_set", feature_set_name)
            mlflow.log_param("model", model_name)
            mlflow.log_param("test_target_year", TEST_TARGET_YEAR)

            for k, v in grid.best_params_.items():
                mlflow.log_param(f"best_{k}", v)

            for k, v in metrics.items():
                mlflow.log_metric(k, v)

            # (선택) 모델 저장
            mlflow.sklearn.log_model(
                best_model,
                artifact_path="model"
            )

            # =========================
            # 6. 결과 테이블용 저장
            # =========================
            row = {
                "feature_set": feature_set_name,
                "model": model_name,
                "n_features": len(feature_cols),
                "best_params": grid.best_params_
            }
            row.update(metrics)

            results.append(row)


In [ ]:
import pandas as pd

results_df = pd.DataFrame(results)

results_df.sort_values(
    by=["top20_precision", "pr_auc"],
    ascending=False
).reset_index(drop=True)


## 모델 정의(Baseline)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

In [ ]:
# =====================================
# Model Definitions
# =====================================

models = {
    "LogReg": LogisticRegression(
        max_iter=1000,
        class_weight="balanced",
        n_jobs=-1
    ),
    "RF": RandomForestClassifier(
        n_estimators=300,
        max_depth=8,
        random_state=42,
        n_jobs=-1
    ),
    "XGB": XGBClassifier(
        n_estimators=300,
        max_depth=5,
        learning_rate=0.05,
        subsample=0.8,
        colsample_bytree=0.8,
        eval_metric="logloss",
        random_state=42,
        n_jobs=-1
    ),
    "LGBM": LGBMClassifier(
        n_estimators=300,
        learning_rate=0.05,
        num_leaves=31,
        random_state=42,
        n_jobs=-1
    )
}

# Baseline
baseline_models = {
    "LogReg": models["LogReg"]
}

# Tree-based models
tree_models = {
    k: v for k, v in models.items()
    if k != "LogReg"
}


## 하이퍼파라미터 튜닝 전 모델 성능

In [ ]:
from sklearn.metrics import roc_auc_score

baseline_results = {}

for name, model in tree_models.items():
    model.fit(X_train, y_train)

    y_pred_proba = model.predict_proba(X_test)[:, 1]
    auc = roc_auc_score(y_test, y_pred_proba)

    baseline_results[name] = auc
    print(f"{name} ROC-AUC: {auc:.4f}")


In [ ]:
from sklearn.metrics import roc_auc_score

logreg = baseline_models["LogReg"]
logreg.fit(X_train, y_train)

y_pred_proba = logreg.predict_proba(X_test)[:, 1]
auc = roc_auc_score(y_test, y_pred_proba)

print(f"LogReg ROC-AUC: {auc:.4f}")


In [ ]:
from sklearn.metrics import (
    average_precision_score,
    f1_score,
    brier_score_loss
)

def evaluate_metrics(y_true, y_pred, y_proba):
    return {
        # PR-AUC
        "pr_auc": average_precision_score(y_true, y_proba),

        # Top-K Precision
        "top20_precision": top_k_precision(y_true, y_proba, k=20),
        "top50_precision": top_k_precision(y_true, y_proba, k=50),

        # Classification balance
        "f1": f1_score(y_true, y_pred),

        # Probability calibration
        "brier": brier_score_loss(y_true, y_proba)
    }


In [ ]:
baseline_results = []

# 1) Tree 모델들
for name, model in tree_models.items():
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)[:, 1]

    metrics = evaluate_metrics(y_test, y_pred, y_proba)

    baseline_results.append({
        "model": name,
        **metrics,
        "stage": "baseline (before tuning)"
    })

# 2) Logistic Regression
logreg = baseline_models["LogReg"]
logreg.fit(X_train, y_train)

y_pred = logreg.predict(X_test)
y_proba = logreg.predict_proba(X_test)[:, 1]

metrics = evaluate_metrics(y_test, y_pred, y_proba)

baseline_results.append({
    "model": "LogReg",
    **metrics,
    "stage": "baseline (before tuning)"
})


In [ ]:
import pandas as pd

baseline_df = pd.DataFrame(baseline_results)

baseline_df = baseline_df.sort_values(
    by="pr_auc", ascending=False
).reset_index(drop=True)

baseline_df


## 하이퍼파라미터 튜닝(우선 보류)

In [ ]:
from sklearn.base import clone
import itertools
import pandas as pd
import numpy as np

def rolling_hyperparam_search(
    model_class,
    param_grid,
    X, y, years,
    start_year=2019,
    end_year=2022
):
    results = []

    param_names = list(param_grid.keys())
    param_combinations = list(itertools.product(*param_grid.values()))

    for params in param_combinations:
        param_dict = dict(zip(param_names, params))
        fold_scores = []

        for train_end in range(start_year, end_year + 1):
            val_year = train_end + 1

            train_mask = years <= train_end
            val_mask   = years == val_year

            X_tr  = X.loc[train_mask]
            y_tr  = y.loc[train_mask]

            X_val = X.loc[val_mask]
            y_val = y.loc[val_mask]


            if X_tr.shape[0] == 0 or X_val.shape[0] == 0:
                continue

            if y_tr.nunique() < 2:
                continue


            model = model_class(**param_dict)
            model.fit(X_tr, y_tr)

            y_pred = model.predict(X_val)
            y_proba = model.predict_proba(X_val)[:, 1]

            score = evaluate_model(y_val, y_pred, y_proba)
            fold_scores.append(score)

        if len(fold_scores) == 0:
            continue

        avg_score = pd.DataFrame(fold_scores).mean().to_dict()
        avg_score.update(param_dict)

        results.append(avg_score)
    
    return pd.DataFrame(results)



## 모델별 튜닝

### LGBM

In [ ]:
from lightgbm import LGBMClassifier

lgbm_param_grid = {
    "n_estimators": [200, 400],
    "learning_rate": [0.03, 0.05],
    "num_leaves": [15, 31],
    "max_depth": [-1, 5],
    "random_state": [42]
}

lgbm_results = rolling_hyperparam_search(
    LGBMClassifier,
    lgbm_param_grid,
    X_train,
    y_train,
    years_train
)

lgbm_results.sort_values("pr_auc", ascending=False).head()


### XGBoost 

In [ ]:
from xgboost import XGBClassifier

xgb_param_grid = {
    "n_estimators": [200, 400],
    "max_depth": [4, 6],
    "learning_rate": [0.03, 0.05],
    "subsample": [0.8],
    "colsample_bytree": [0.8],
    "eval_metric": ["logloss"],
    "random_state": [42]
}

xgb_results = rolling_hyperparam_search(
    XGBClassifier,
    xgb_param_grid,
    X_train,
    y_train,
    years_train
)

xgb_results.sort_values("pr_auc", ascending=False).head()


### RF

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_param_grid = {
    "n_estimators": [200, 400],
    "max_depth": [5, 10],
    "min_samples_leaf": [1, 3],
    "random_state": [42],
    "n_jobs": [-1]
}

rf_results = rolling_hyperparam_search(
    RandomForestClassifier,
    rf_param_grid,
    X_train,
    y_train,
    years_train
)

rf_results.sort_values("pr_auc", ascending=False).head()


### LR(Logistic Regression)

In [ ]:
from sklearn.linear_model import LogisticRegression

lr_param_grid = {
    "C": [0.1, 1.0, 5.0],
    "penalty": ["l2"],
    "solver": ["lbfgs"],
    "max_iter": [1000]
}

lr_results = rolling_hyperparam_search(
    LogisticRegression,
    lr_param_grid,
    X_train,        
    y_train,        
    years_train
)

# PR-AUC 기준으로 정렬 (핵심 수정)
lr_results.sort_values("pr_auc", ascending=False).head()


## ML Flow

In [ ]:
# =========================================================
# 0. 라이브러리
# =========================================================
from datetime import datetime
import pandas as pd
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

import mlflow
import mlflow.sklearn


# =========================================================
# 1. 모델 설정
# =========================================================
model_configs = {
    "LogReg": (
        LogisticRegression,
        {
            "C": [0.1, 1.0, 5.0],
            "penalty": ["l2"],
            "solver": ["lbfgs"],
            "max_iter": [1000]
        }
    ),
    "RF": (
        RandomForestClassifier,
        {
            "n_estimators": [200, 400],
            "max_depth": [5, 10],
            "min_samples_leaf": [1, 3],
            "random_state": [42],
            "n_jobs": [-1]
        }
    ),
    "XGB": (
        XGBClassifier,
        {
            "n_estimators": [200, 400],
            "max_depth": [4, 6],
            "learning_rate": [0.03, 0.05],
            "subsample": [0.8],
            "colsample_bytree": [0.8],
            "eval_metric": ["logloss"],
            "random_state": [42]
        }
    ),
    "LGBM": (
        LGBMClassifier,
        {
            "n_estimators": [200, 400],
            "learning_rate": [0.03, 0.05],
            "num_leaves": [15, 31],
            "max_depth": [-1, 5],
            "random_state": [42]
        }
    )
}


# =========================================================
# 2. MLflow 설정
# =========================================================
mlflow.set_experiment("growth_prediction_rolling")

results = []


# =========================================================
# 3. 모델별 롤링 튜닝 + 로깅
# =========================================================
for model_name, (model_class, param_grid) in model_configs.items():

    run_ts = datetime.now().strftime("%Y%m%d_%H%M%S")
    print(f"\n=== {model_name} rolling tuning ===")

    with mlflow.start_run(
        run_name=f"{model_name}_rolling_{run_ts}"
    ):

        df_results = rolling_hyperparam_search(
            model_class=model_class,
            param_grid=param_grid,
            X=X_train,
            y=y_train,
            years=years_train,
            start_year=2019,
            end_year=2022
        )

        # ✅ PR-AUC 기준으로 best 선택
        best_row = df_results.sort_values(
            "pr_auc", ascending=False
        ).iloc[0]

        best_params = {k: best_row[k] for k in param_grid.keys()}

        # 타입 정리
        for k, v in best_params.items():
            if isinstance(v, float) and k not in ["learning_rate"]:
                best_params[k] = int(v)

        # MLflow 로깅
        mlflow.log_params(best_params)

        mlflow.log_metrics({
            "pr_auc": best_row["pr_auc"],
            "top20_precision": best_row["top20_precision"],
            "top50_precision": best_row["top50_precision"],
            "f1": best_row["f1"],
            "brier": best_row["brier"]
        })

        # 결과 CSV 아티팩트
        result_path = f"{model_name}_rolling_results_{run_ts}.csv"
        df_results.to_csv(result_path, index=False)
        mlflow.log_artifact(result_path)

        results.append({
            "model": model_name,
            **best_params,
            "pr_auc": best_row["pr_auc"],
            "top20_precision": best_row["top20_precision"],
            "top50_precision": best_row["top50_precision"],
            "f1": best_row["f1"],
            "brier": best_row["brier"]
        })


# =========================================================
# 4. 전체 결과 요약 테이블
# =========================================================
summary_df = pd.DataFrame(results).sort_values(
    "pr_auc", ascending=False
)

summary_df


## 모델별 최고 성능 비교 
- ROC-AUC 최댓값 기준

In [ ]:
best_rf   = rf_results.sort_values("pr_auc", ascending=False).iloc[0]
best_lgbm = lgbm_results.sort_values("pr_auc", ascending=False).iloc[0]
best_xgb  = xgb_results.sort_values("pr_auc", ascending=False).iloc[0]
best_lr  = lr_results.sort_values("pr_auc", ascending=False).iloc[0]


In [ ]:
compare_df = pd.DataFrame([
    {"model": "RF",   **best_rf},
    {"model": "LGBM", **best_lgbm},
    {"model": "XGB",  **best_xgb},
    {"model": "LR",  **best_lr},
])

compare_df = compare_df[
    ["model", "pr_auc", "top20_precision", "top50_precision", "f1", "brier"]
]

compare_df.sort_values("pr_auc", ascending=False)

# LGBM 선정

### 최종 모델 학습, 테스트 (2023 -> 2024) 

In [ ]:
# best_params = lgbm_results.sort_values("roc_auc", ascending=False).iloc[0].to_dict()

# # 🔥 int여야 하는 파라미터 강제 변환
# int_params = ["random_state", "seed", "n_estimators", "num_leaves", "max_depth"]

# for p in int_params:
#     if p in best_params and best_params[p] is not None:
#         best_params[p] = int(best_params[p])

# best_model = LGBMClassifier(**best_params)




# best_model.fit(X_train_full, y_train_full)

# y_pred_test = best_model.predict(X_test)
# y_proba_test = best_model.predict_proba(X_test)[:, 1]

# test_metrics = evaluate_model(y_test, y_pred_test, y_proba_test)
# test_metrics


In [ ]:
from lightgbm import LGBMClassifier

# 1️⃣ PR-AUC 기준으로 best 파라미터 선택
best_params = (
    lgbm_results
    .sort_values("pr_auc", ascending=False)
    .iloc[0]
    .to_dict()
)

# 2️⃣ metric 컬럼 제거 (중요)
metric_cols = ["pr_auc", "top20_precision", "top50_precision", "f1", "brier"]
for m in metric_cols:
    best_params.pop(m, None)

# 3️⃣ int 타입 강제 변환
int_params = ["random_state", "seed", "n_estimators", "num_leaves", "max_depth"]

for p in int_params:
    if p in best_params and best_params[p] is not None:
        best_params[p] = int(best_params[p])

# 4️⃣ Best 모델 정의
best_model = LGBMClassifier(**best_params)

# 5️⃣ Train 구간 전체로 재학습 (2020–2023)
best_model.fit(X_train, y_train)

# 6️⃣ Final Test (2024)
y_pred_test = best_model.predict(X_test)
y_proba_test = best_model.predict_proba(X_test)[:, 1]

test_metrics = evaluate_model(y_test, y_pred_test, y_proba_test)
test_metrics


## 검증

### Hold-Out 
> - Train: 2019–2022
> - Test : 2023 (→ 2024 성장 여부)

In [ ]:
def holdout_validation(model, X, y, years):
    train_mask = years <= 2021
    val_mask   = years == 2022

    X_tr, y_tr = X[train_mask], y[train_mask]
    X_val, y_val = X[val_mask], y[val_mask]

    model.fit(X_tr, y_tr)
    y_pred = model.predict(X_val)
    y_proba = model.predict_proba(X_val)[:, 1]

    return evaluate_model(y_val, y_pred, y_proba)


### GroupKFold 검증
- 같은 기업이 Train / Test 에 동시에 등장 x

In [ ]:
from sklearn.model_selection import GroupKFold

def group_kfold_validation(model, X, y, groups, n_splits=5):
    gkf = GroupKFold(n_splits=n_splits)
    scores = []

    for train_idx, val_idx in gkf.split(X, y, groups):
        X_tr, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_tr, y_val = y.iloc[train_idx], y.iloc[val_idx]

        model.fit(X_tr, y_tr)
        y_pred = model.predict(X_val)
        y_proba = model.predict_proba(X_val)[:, 1]

        scores.append(evaluate_model(y_val, y_pred, y_proba))

    return pd.DataFrame(scores).mean().to_dict()


### Rolling Validation

In [ ]:
def rolling_validation(model, X, y, years):
    fold_scores = []

    for train_end in sorted(years.unique()):
        val_year = train_end + 1

        train_mask = years <= train_end
        val_mask   = years == val_year

        # ✅ pandas 기준 slicing (중요)
        X_tr  = X.loc[train_mask]
        y_tr  = y.loc[train_mask]
        X_val = X.loc[val_mask]
        y_val = y.loc[val_mask]

        # 🔥 방어 1: 샘플 수
        if X_tr.shape[0] == 0 or X_val.shape[0] == 0:
            continue

        # 🔥 방어 2: train에 클래스 2개 이상
        if y_tr.nunique() < 2:
            continue

        # 🔥 방어 3: val에 클래스 존재
        if y_val.nunique() < 1:
            continue

        model_clone = clone(model)
        model_clone.fit(X_tr, y_tr)

        y_pred  = model_clone.predict(X_val)
        y_proba = model_clone.predict_proba(X_val)[:, 1]

        fold_scores.append(
            evaluate_model(y_val, y_pred, y_proba)
        )

    if len(fold_scores) == 0:
        return None

    return pd.DataFrame(fold_scores).mean().to_dict()


## 모델별 전부 실행

In [ ]:
all_results = {}

# 🔥 트리 모델만 3종 검증
for name, model in tree_models.items():
    print(f"\n=== {name} ===")

    all_results[name] = {
        "holdout": holdout_validation(
            model,
            X_train,
            y_train,
            years_train
        ),
        "groupkfold": group_kfold_validation(
            model,
            X_train,
            y_train,
            groups_train
        ),
        "rolling": rolling_validation(
            model,
            X_train,
            y_train,
            years_train
        )
    }


In [ ]:
all_results["LGBM"]["rolling"]


In [ ]:
rows = []

for model_name, validations in all_results.items():
    for val_type, metrics in validations.items():

        if metrics is None:
            continue

        row = {
            "model": model_name,
            "validation": val_type
        }
        row.update(metrics)

        rows.append(row)

results_df = pd.DataFrame(rows)
results_df


In [ ]:
import mlflow
from datetime import datetime

run_ts = datetime.now().strftime("%Y%m%d_%H%M%S")

with mlflow.start_run(
    run_name=f"validation_summary_{run_ts}"
):

    # 1️⃣ 전체 결과 CSV 저장
    summary_path = f"validation_results_summary_{run_ts}.csv"
    results_df.to_csv(summary_path, index=False)
    mlflow.log_artifact(summary_path)

    # 2️⃣ 핵심 metric 요약 로깅
    for _, row in results_df.iterrows():
        prefix = f"{row['model']}_{row['validation']}"

        mlflow.log_metric(f"{prefix}_pr_auc", row["pr_auc"])
        mlflow.log_metric(f"{prefix}_top20_precision", row["top20_precision"])
        mlflow.log_metric(f"{prefix}_top50_precision", row["top50_precision"])
        mlflow.log_metric(f"{prefix}_f1", row["f1"])
        mlflow.log_metric(f"{prefix}_brier", row["brier"])


In [ ]:
key_metrics = [
    "pr_auc",
    "top20_precision",
    "top50_precision",
    "f1",
    "brier"
]

compare_table = (
    results_df
    .pivot_table(
        index="model",
        columns="validation",
        values=key_metrics
    )
    .round(3)
)

compare_table


- ① 검증에서 고른 best_params로
- ② 2023년까지 모든 데이터로 재학습하고
- ③ 2024년 말 피처를 넣어
- ④ 2025년 성장 확률을 예측한다

# Feature Importance
- 모델의 예측 결과에 각 피처가 기여한 상대적 중요도
    - 값이 클수록 피처 여부에 따라 판단이 많이 바뀜
    - 값이 작을수록 있어도 그만 없어도 그만

In [ ]:
import pandas as pd

# model이 학습에 사용한 feature 기준
feature_cols = model.feature_names_in_

feature_importance = pd.DataFrame({
    'feature': feature_cols,
    'importance': model.feature_importances_
}).sort_values(by='importance', ascending=False)

feature_importance.head(20)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# =====================================
# 1. 히트맵 대상 피처
# =====================================
corr_cols = [
    # 기존 정량
    'patent_count_5y',
    'patent_count_recent',
    'ipc_diversity',
    'patent_register_rate',
    'patent_citation_total',
    'patent_citation_avg',
    'patent_citation_age_adj_total',
    'patent_citation_age_adj_avg',
    'no_patent_flag',

    # # 🔥 변화형 피처
    # 'patent_recent_ratio',
    # 'citation_recent_ratio',
    # 'patent_growth_rate',
    # 'patent_growth_accel',
    # 'new_ipc_ratio',
    # 'recent_ipc_overlap',
    # 'citation_per_patent_recent'
]

# =====================================
# 2. 상관계수 계산
# =====================================
corr_matrix = feature_df[corr_cols].corr()

# =====================================
# 3. 보기 좋게 상삼각만 표시 (선택)
# =====================================
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))

# =====================================
# 4. 히트맵 그리기 (숫자 포함)
# =====================================
plt.figure(figsize=(16, 14))
sns.heatmap(
    corr_matrix,
    mask=mask,              # 👈 상삼각 숨기기 (선택)
    cmap="coolwarm",
    center=0,
    annot=True,             # 👈 숫자 표시
    fmt=".2f",              # 👈 소수점 2자리
    annot_kws={"size": 9},  # 👈 숫자 크기
    linewidths=0.5,
    cbar_kws={"shrink": 0.8}
)

plt.title("Feature Correlation Heatmap (with values)", fontsize=15)
plt.tight_layout()
plt.show()
